In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 17 15:30:35 2020

@author: Khari Jarrett
"""

'''
All code is highly based on Ildoo Kim's code (https://github.com/ildoonet/tf-openpose)
and derived from the OpenPose Library (https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/LICENSE)
'''
import tensorflow as tf
import cv2
import numpy as np
import argparse
import csv
import os
from tensorflow.python.platform import gfile


from common import estimate_pose, draw_humans, read_imgfile

import time

from IPython.display import clear_output, Image, display, HTML



In [2]:
import random
import math

def make_ellipse(image, row):
        
        center = (int(round(row[0])),int(round(row[1])))
        axes = (int(round(row[2])),int(round(row[3])))
        color = (255,255,255)
        angle = int(round(row[4] * 180 / math.pi))
                      

        return cv2.ellipse(image, center, axes, angle, 0, 360, color, -1)     
    
    
def makeMask(tubefile, numframes, in_height=720, in_width=1280):
    #Later we will update to handle CSVs and txt files, but be cool....

    THEMASK = np.zeros((in_height, in_width,3,numframes), dtype=np.uint8)

    D = np.genfromtxt(tubefile) #Got the data!
    
    for f in range(0,numframes):
        frame = np.zeros((in_height, in_width,3), dtype=np.uint8)
        
        d = [row for row in D if row[5]==f] #list of rows this frame
        for row in d:
            frame = make_ellipse(frame,row)
        
        THEMASK[:,:,:,f] = frame
        cv2.imwrite("masktest.jpg", frame)
        cv2.imshow("image", frame)
        cv2.waitKey(25)
        
    return THEMASK
        

    
A = makeMask("int6_tubelist.txt", 10)
print(type(A))

<class 'numpy.ndarray'>


In [19]:

def runvid(vidpath):
    # The plan is to import a video, split it into images and then feed them into the run() function
    vidcap = cv2.VideoCapture(vidpath)
    vidbase = os.path.split(vidpath)[1].split(".")[0]

    i=1
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        cv2.imwrite(vidbase+'_frame'+str(i)+'.jpg',frame)
        i=i+1
   
    vidcap.release()
    cv2.destroyAllWindows()
    lastframe=i
    
    humanlist = [0] * lastframe
    framelist = [0] * lastframe
    
    #DOPENESS. NOW GET IT TO SAVE EVERYTHING IMPORTANT
    #frame_width = int(vidcap.get(3))
    #frame_height = int(vidcap.get(4))
   
    
    vidfile = vidbase + '_OPoutmask3.mp4'
    csvfile = open(vidbase + '_OPCSVmask3.csv','w')
    tubefile = vidbase + '_tubelist.txt'

    mask = makeMask(tubefile, lastframe)
    HL,img = run(vidbase+'_frame1.jpg',mask[:,:,:,1])
    
    
    #Look at imageshape output and take size from there (reversed)
    vidwriter = cv2.VideoWriter(vidfile, cv2.VideoWriter_fourcc(*'MP4V'),10, (img.shape[1],img.shape[0] ))
    csvwriter = csv.writer(csvfile)


    for i in range(1, lastframe):
        print("FRAME" , i,)
        maskslice = mask[:,:,:,i]     #HxWx3
        humanlist[i], image = run(vidbase+'_frame'+str(i)+'.jpg', maskslice)
        
        #Draw mask on video
        maskslice = maskslice[:,:,0]
        maskslice = cv2.resize(maskslice, dsize=(image.shape[1],image.shape[0]), interpolation = cv2.INTER_CUBIC)
        print("Mask:",maskslice.shape)
        print("Image:", image.shape)
        
        for j in range(0,maskslice.shape[0]):
            for k in range(0,maskslice.shape[1]):
                if maskslice[j,k] == 255:
                    image[j,k,0] = 255
        
        
        vidwriter.write(image)

      #  if i==75 or i==76 or i==77:
      #      cv2.imwrite(vidbase+'_outframe'+str(i)+'.jpg',frame)
        #cv2.imshow('frame',image)
        #cv2.waitKey(25)
        #framelist[i] = image
        
        for j in range(0,len(humanlist[i])):
            for key, val in humanlist[i][j].items():
                csvwriter.writerow([i,j,key,val])
            
        
    vidwriter.release()
    cv2.destroyAllWindows()
    csvfile.close()
    
    
def run(imgpath, mask, input_width=656, input_height=368):
    
    t0 = time.time()

    tf.compat.v1.reset_default_graph()
    
    from tensorflow.core.framework import graph_pb2
    graph_def = graph_pb2.GraphDef()
    # Download model from https://www.dropbox.com/s/2dw1oz9l9hi9avg/optimized_openpose.pb
    with open('models/optimized_openpose.pb', 'rb') as f:
        graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

    t1 = time.time()
    print(t1 - t0)

    inputs = tf.compat.v1.get_default_graph().get_tensor_by_name('inputs:0')
    heatmaps_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('Mconv7_stage6_L2/BiasAdd:0')
    pafs_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('Mconv7_stage6_L1/BiasAdd:0')
    
    
    t2 = time.time()
    print(t2 - t1)

    image = read_imgfile(imgpath, input_width, input_height)

    t3 = time.time()
    print(t3 - t2)

    with tf.compat.v1.Session() as sess:
        #writer = tf.summary.FileWriter('output', tf.get_default_graph())
        heatMat, pafMat = sess.run([heatmaps_tensor, pafs_tensor], feed_dict={
            inputs: image
        })
                
        #writer.close()
        t4 = time.time()
        print(t4 - t3)

        
        heatMat, pafMat = heatMat[0], pafMat[0]
        #The shape is (height x width x n_parts(19/38))
 

        #RIGHT SIDE OFF MASK
        #mask = np.zeros(heatMat.shape)
        #mask[:, 0:round(heatMat.shape[1]/2), :] = np.ones((heatMat.shape[0], round(heatMat.shape[1]/2), heatMat.shape[2]))
        
        #BOTTOM SIDE OFF MASK
        #mask = np.zeros(heatMat.shape)
        #mask[0:round(heatMat.shape[0]/2), :,:] = np.ones((round(heatMat.shape[0]/2), heatMat.shape[1] , heatMat.shape[2]))
        
        #Mask handling
        Mask = np.zeros((heatMat.shape[0],heatMat.shape[1],19))
        mask = mask[:,:,0]
        mask = cv2.resize(mask, dsize=(heatMat.shape[1],heatMat.shape[0]), interpolation = cv2.INTER_CUBIC)
        for i in range(0,19):
            Mask[:,:,i] = mask
        heatMat = np.multiply(Mask,heatMat)
                
        humans = estimate_pose(heatMat, pafMat)

        # display
        image = cv2.imread(imgpath)
        image_h, image_w = image.shape[:2]
        image = draw_humans(image, humans)

        scale = 480.0 / image_h
        newh, neww = 480, int(scale * image_w + 0.5)
        
        image = cv2.resize(image, (neww, newh), interpolation=cv2.INTER_AREA)
        print("Image shape: ", image.shape)
        
        cv2.imshow('result', image)
        t5 = time.time()
        print(t5 - t4)
        cv2.waitKey(25)
        
        #show_graph(tf.compat.v1.get_default_graph().as_graph_def())
        

    return humans, image

In [20]:
runvid("int6.mp4")

1.8006620407104492
0.0
0.03516840934753418
10.949174165725708
Image shape:  (480, 853, 3)
0.09821391105651855
FRAME 1
2.3517239093780518
0.000997304916381836
0.030953168869018555
9.58249282836914
Image shape:  (480, 853, 3)
0.07879495620727539
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 2
1.9342668056488037
0.0
0.0329136848449707
10.131792306900024
Image shape:  (480, 853, 3)
0.06103372573852539
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 3
1.706491231918335
0.0
0.04288125038146973
10.120834112167358
Image shape:  (480, 853, 3)
0.06512022018432617
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 4
1.884462594985962
0.0
0.034873008728027344
10.056137561798096
Image shape:  (480, 853, 3)
0.08299899101257324
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 5
1.7375342845916748
0.0
0.03792381286621094
9.938191890716553
Image shape:  (480, 853, 3)
0.13527655601501465
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 6
2.0356686115264893
0.0
0.031905174255371094
10.303946495056152
Image shape:  (480, 853, 3

Mask: (480, 853)
Image: (480, 853, 3)
FRAME 52
1.8948774337768555
0.0
0.046875715255737305
10.376190423965454
Image shape:  (480, 853, 3)
0.08911013603210449
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 53
2.0446155071258545
0.0
0.04192495346069336
10.592222452163696
Image shape:  (480, 853, 3)
0.07834959030151367
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 54
1.985107660293579
0.0
0.05385613441467285
10.055038452148438
Image shape:  (480, 853, 3)
0.0951383113861084
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 55
1.7373406887054443
0.0
0.03590130805969238
9.779122352600098
Image shape:  (480, 853, 3)
0.10295391082763672
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 56
2.251056671142578
0.0
0.046025753021240234
10.70921540260315
Image shape:  (480, 853, 3)
0.09940934181213379
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 57
1.9029796123504639
0.0
0.03789806365966797
12.147730588912964
Image shape:  (480, 853, 3)
0.11413264274597168
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 58
1.824173688888

11.190527439117432
Image shape:  (480, 853, 3)
0.3336629867553711
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 103
1.8786916732788086
0.0009980201721191406
0.03789830207824707
11.210349559783936
Image shape:  (480, 853, 3)
0.11286020278930664
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 104
1.9090733528137207
0.0
0.04296445846557617
10.094921350479126
Image shape:  (480, 853, 3)
0.18467068672180176
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 105
2.0097079277038574
0.0
0.0339510440826416
9.785804033279419
Image shape:  (480, 853, 3)
0.09126472473144531
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 106
2.062737464904785
0.0
0.0384066104888916
9.992770195007324
Image shape:  (480, 853, 3)
0.11890220642089844
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 107
1.8196659088134766
0.0
0.034906864166259766
9.59149169921875
Image shape:  (480, 853, 3)
0.10141992568969727
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 108
1.7538037300109863
0.0
0.04229855537414551
10.651349782943726
Image shape:  (480, 85

FRAME 153
1.9140777587890625
0.0
0.036897897720336914
9.894013166427612
Image shape:  (480, 853, 3)
0.6941437721252441
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 154
1.8361940383911133
0.0
0.03789925575256348
10.65907883644104
Image shape:  (480, 853, 3)
7.560123443603516
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 155
1.7776830196380615
0.0
0.036859989166259766
10.35959243774414
Image shape:  (480, 853, 3)
8.884884119033813
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 156
1.7752881050109863
0.0
0.03989267349243164
9.709625720977783
Image shape:  (480, 853, 3)
8.463067293167114
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 157
2.004523992538452
0.0
0.03386044502258301
10.018532276153564
Image shape:  (480, 853, 3)
3.2658910751342773
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 158
1.8359427452087402
0.0
0.03494691848754883
9.692453861236572
Image shape:  (480, 853, 3)
3.1514034271240234
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 159
1.903578519821167
0.0
0.03890109062194824
11.1143531799

Image shape:  (480, 853, 3)
11.723844289779663
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 205
1.79073166847229
0.0
0.037899017333984375
10.907989025115967
Image shape:  (480, 853, 3)
5.2732813358306885
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 206
2.417206287384033
0.0
0.041852474212646484
10.59975266456604
Image shape:  (480, 853, 3)
5.792726278305054
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 207
1.9386277198791504
0.0
0.03789806365966797
11.365535497665405
Image shape:  (480, 853, 3)
4.365443706512451
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 208
2.0758590698242188
0.0
0.03989434242248535
10.530466079711914
Image shape:  (480, 853, 3)
3.85304594039917
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 209
1.8616962432861328
0.0
0.035856008529663086
11.742565393447876
Image shape:  (480, 853, 3)
3.5456912517547607
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 210
1.9990174770355225
0.0009982585906982422
0.04193711280822754
10.099576473236084
Image shape:  (480, 853, 3)
3.971509933471679

...

FRAME 306
1.8210704326629639
0.0
0.03790020942687988
10.155001640319824
Image shape:  (480, 853, 3)
13.367584705352783
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 307
1.9471440315246582
0.0009970664978027344
0.035944223403930664
10.38693881034851
Image shape:  (480, 853, 3)
10.570025205612183
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 308
1.9574005603790283
0.0
0.0538477897644043
10.960762739181519
Image shape:  (480, 853, 3)
8.504181146621704
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 309
2.525782585144043
0.0009987354278564453
0.048868656158447266
10.819648027420044
Image shape:  (480, 853, 3)
8.720287322998047
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 310
1.9261775016784668
0.0
0.039893150329589844
10.545753240585327
Image shape:  (480, 853, 3)
10.422144651412964
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 311
2.1638684272766113
0.0
0.041280269622802734
10.600301265716553
Image shape:  (480, 853, 3)
9.660224676132202
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 312
1.801760911941

FRAME 357
1.9355063438415527
0.0
0.03690147399902344
9.98722243309021
Image shape:  (480, 853, 3)
5.308898210525513
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 358
1.8626315593719482
0.0
0.047922611236572266
10.248589277267456
Image shape:  (480, 853, 3)
12.838850736618042
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 359
1.937577486038208
0.0
0.035903215408325195
10.37612795829773
Image shape:  (480, 853, 3)
13.177557706832886
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 360
1.9366049766540527
0.0
0.038945913314819336
10.382611513137817
Image shape:  (480, 853, 3)
12.777700901031494
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 361
2.0483195781707764
0.0
0.04487919807434082
10.386659145355225
Image shape:  (480, 853, 3)
6.610685110092163
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 362
2.142136812210083
0.000997304916381836
0.04288482666015625
10.830911636352539
Image shape:  (480, 853, 3)
6.075171232223511
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 363
1.861673355102539
0.0
0.038854837417

FRAME 408
1.751544713973999
0.0
0.039894819259643555
9.68491816520691
Image shape:  (480, 853, 3)
0.04840850830078125
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 409
2.2122552394866943
0.0
0.0359039306640625
11.426482915878296
Image shape:  (480, 853, 3)
0.14864706993103027
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 410
2.20065975189209
0.000997304916381836
0.036448001861572266
10.29053282737732
Image shape:  (480, 853, 3)
0.10372209548950195
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 411
1.814333200454712
0.0
0.04186868667602539
9.808352708816528
Image shape:  (480, 853, 3)
0.06390047073364258
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 412
1.8479022979736328
0.0
0.03897523880004883
9.526180744171143
Image shape:  (480, 853, 3)
0.07294487953186035
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 413
1.807304859161377
0.0
0.0348665714263916
10.381489992141724
Image shape:  (480, 853, 3)
0.07930850982666016
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 414
1.774592399597168
0.0
0.03989434242

1.8451340198516846
0.0009982585906982422
0.04288601875305176
10.19505763053894
Image shape:  (480, 853, 3)
0.14914345741271973
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 460
1.9999959468841553
0.0009970664978027344
0.03513169288635254
10.633331298828125
Image shape:  (480, 853, 3)
0.16750478744506836
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 461
1.7860629558563232
0.0
0.03490710258483887
10.008826732635498
Image shape:  (480, 853, 3)
0.1755070686340332
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 462
1.8687975406646729
0.0
0.03591561317443848
9.793815612792969
Image shape:  (480, 853, 3)
0.149641752243042
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 463
1.914785385131836
0.0
0.0458376407623291
10.24446702003479
Image shape:  (480, 853, 3)
0.2520594596862793
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 464
2.039287805557251
0.0
0.04487895965576172
10.724714279174805
Image shape:  (480, 853, 3)
0.23712992668151855
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 465
1.972879409790039
0.0
0.03

Mask: (480, 853)
Image: (480, 853, 3)
FRAME 510
1.8756320476531982
0.0
0.03291153907775879
9.853819608688354
Image shape:  (480, 853, 3)
0.12997841835021973
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 511
1.779994010925293
0.0
0.04092550277709961
9.720707654953003
Image shape:  (480, 853, 3)
0.13546347618103027
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 512
1.7997732162475586
0.0
0.03490614891052246
9.962584733963013
Image shape:  (480, 853, 3)
0.16968631744384766
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 513
1.9348034858703613
0.0
0.038895606994628906
11.602492332458496
Image shape:  (480, 853, 3)
0.219010591506958
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 514
1.9765279293060303
0.0
0.11568999290466309
10.667465925216675
Image shape:  (480, 853, 3)
0.2797536849975586
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 515
2.1156911849975586
0.0
0.03889584541320801
10.007554054260254
Image shape:  (480, 853, 3)
0.221726655960083
Mask: (480, 853)
Image: (480, 853, 3)
FRAME 516
1.8330402374

In [67]:
#If it's real, make a mask video.

maskfile = "int6_maskout.mp4"
A = makeMask("int6_tubelist.txt", 540)
vidwriter = cv2.VideoWriter(maskfile, cv2.VideoWriter_fourcc(*'mp4v'),10, (1280,720 ))

for i in range(0,540):
    frame = A[:,:,:,i]
    cv2.imwrite("masktest.jpg", frame)
    vidwriter.write(frame)
    
vidwriter.release()
cv2.destroyAllWindows()
     
#AYE LMAO IT ACTUALLY WORKS

In [59]:

print(np.shape(A))
A2 = np.tile(A, [ 1, 2] )
print(np.shape(A2))


(50, 100)
(50, 200)


In [61]:
runvid('int2.mp4')

2.8805041313171387
0.0
0.09262943267822266
Heatmat shape: (1, 46, 82, 19)
PAFMat shape: (1, 46, 82, 38)
13.021999597549438
Heatmat shape: (46, 82, 19)
PAFMat shape: (46, 82, 38)
Mask (46, 82, 19)
Image shape:  (480, 637, 3)
0.09077620506286621
FRAME 70
2.015958309173584
0.0009992122650146484
0.06212782859802246
Heatmat shape: (1, 46, 82, 19)
PAFMat shape: (1, 46, 82, 38)
10.755788326263428
Heatmat shape: (46, 82, 19)
PAFMat shape: (46, 82, 38)
Mask (46, 82, 19)
Image shape:  (480, 637, 3)
0.10041475296020508
FRAME 71
3.64422345161438
0.0
0.0859076976776123
Heatmat shape: (1, 46, 82, 19)
PAFMat shape: (1, 46, 82, 38)
12.644611120223999
Heatmat shape: (46, 82, 19)
PAFMat shape: (46, 82, 38)
Mask (46, 82, 19)
Image shape:  (480, 637, 3)
0.11274552345275879
FRAME 72
2.956153631210327
0.0
0.11554574966430664
Heatmat shape: (1, 46, 82, 19)
PAFMat shape: (1, 46, 82, 38)
12.97613525390625
Heatmat shape: (46, 82, 19)
PAFMat shape: (46, 82, 38)
Mask (46, 82, 19)
Image shape:  (480, 637, 3)
0.107

In [30]:
type(-1)

int